<h1> 8장 오차 역전파 </h1>
<br> 신경망 내부의 가중치는 경사 하강법의 확장 개념인 오차 역전파 방법을 사용해 수정!<br>
임의의 가중치를 선언하고 결과를 이용해 오차를 구한 뒤 이 오차가 최소인 지점(미분했을 때 기울기 0)으로 계속 조금씩 이동.<br>
다층 퍼셉트론 역시 결괏값의 오차를 구해 이를 토대로 하나 앞선 가중치를 거슬러 올라가 조정!<br>
그러다 보니 최적화의 계산 방향이 출력층에서 시작해 앞으로 진행됨!<br>
그래서 다층 퍼셉트론에서의 최적화 과정을 오차 역전파(Back Propagation)이라 한다.

<h3> 1. 임의의 초기 가중치(w)를 준 뒤 결과(Yout)을 계산. </h3>
<h3> 2. 계산 결과와 우리가 원하는 값 사이의 오차를 계산. </h3>
<h3> 3. 경사 하강법을 이용해 바로 앞 가중치를 오차가 작아지는 방향으로 업데이트. </h3>
<h3> 4. 오차가 최소가 될 때까지 반복. </h3>


In [63]:
import numpy as np
import random
random.seed(777)

data = [
    [[0, 0], [0]],
    [[0, 1], [1]],
    [[1, 0], [1]],
    [[1, 1], [0]]
]
iterations = 5000
lr = 0.1 
mo = 0.9

우리가 원하는 입력 값과 타깃 값을 data 안에 넣어 놓고 실행 횟수, 학습률, 모멘텀 계수를 정함.<br>
모멘텀 계수란 Momentum optimizer를 할 때 필요한 계수인데 경사하강법을 할 때 학습률으로 오차를 줄어나가며 오차가 최소인 점을 찾는 것처럼 
모멘텀 최적화를 할 때는 관성을 받아 경사 하강법 보다 더 빠르게 오차가 최소인 점을 찾는다. 그때 모멘텀 계수가 필요함!


In [64]:
def sigmoid(x, derivative = False):
    if (derivative == True):
        return x * (1-x)
    return 1 / (1 + np.exp(-x))

<h3>시그모이드 함수로 미분을 할 때 얻는 리턴값과 안 할때 얻는 리턴값 설정</h3>

In [65]:
def tanh(x, derivative = False):
    if (derivative == True):
        return 1 - x**2
    return np.tanh(x)

<h3>탄젠트 함수 설정</h3>

In [66]:
def makeMatrix(i, j, fill = 0.0):
    mat = []
    for i in range(i):
        mat.append([fill]*j)
    return mat    

<h3>가중치 배열을 만드는 함수 설정</h3>

In [67]:
class NeuralNetwork:
    
    def __init__(self, num_x, num_yh, num_y0, bias=1):
        self.num_x = num_x + bias
        self.num_yh = num_yh
        self.num_y0 = num_y0
        self.activation_input = [1.0] * self.num_x
        self.activation_hidden = [1.0] * self.num_yh
        self.activation_out = [1.0] * self.num_y0
        self.weight_in = makeMatrix(self.num_x, self.num_yh)
        for i in range(self.num_x):
            for j in range(self.num_yh):
                self.weight_in[i][j] = random.random()
        self.weight_out = makeMatrix(self.num_yh, self.num_y0)
        for j in range(self.num_yh):
            for k in range(self.num_y0):
                self.weight_out[j][k] = random.random()
                
        self.gradient_in = makeMatrix(self.num_x, self.num_yh)
        self.gradient_out = makeMatrix(self.num_yh, self.num_y0)
        
    def update(self, inputs):
    
        for i in range(self.num_x -1):
            self.activation_input[i] = inputs[i]
        for j in range(self.num_yh):
            sum = 0.0
            for i in range(self.num_x):
                sum = sum + self.activation_input[i] * self.weight_in[i][j]
            self.activation_hidden[j] = tanh(sum, False)
        for k in range(self.num_y0):
            sum = 0.0
            for j in range(self.num_yh):
                sum = sum + self.activation_hidden[j] + self.weight_out[j][k]
            self.activation_out[k] = tanh(sum, False)

        return self.activation_out[:]
    
    def backPropagate(self, targets):
        output_deltas = [0.0] * self.num_y0
        for k in range(self.num_y0):
            error = targets[k] - self.activation_out[k]
            output_deltas[k] = tanh(self.activation_out[k], True) * error
        hidden_deltas = [0.0] * self.num_yh
        for j in range(self.num_yh):
            error = 0.0
            for k in range(self.num_y0):
                error = error + output_deltas[k] * self.weight_out[j][k]
            hidden_deltas[j] = tanh(self.activation_hidden[j], True) * error
        for j in range(self.num_yh):
            for k in range(self.num_y0):
                gradient = output_deltas[k] * self.activation_hidden[j]
                v = mo * self.gradient_in[j][k] - lr * gradient
                self.weight_in[j][k] += v
                self.gradient_out[j][k] = gradient
        for i in range(self.num_x):
            for j in range(self.num_yh):
                gradient = hidden_deltas[j] * self.activation_input[i]
                v = mo * self.gradient_in[i][j] - lr * gradient
                self.weight_in[i][j] += v
                self.gradient_in[i][j] = gradient
        error = 0.0
        for k in range(len(targets)):
            error = error + 0.5 * (targets[k] - self.activation_out[k]) ** 2
        return error
        
    def train(self, patterns):
        for i in range(iterations):
            error = 0.0
            for p in patterns:
                inputs = p[0]
                targets = p[1]
                self.update(inputs)
                error = error + self.backPropagate(targets)
            if i % 500 == 0:
                print("Error : %-.5f"% error)

    def result(self, patterns):
        for p  in patterns:
            print('Input: %s, Predict : %s' %(p[0], self.update(p[0])))

신경망을 실행하는 클래스는 초깃값 지정, 업데이트 함수, 역전파 함수로 구성!<br>
입력값(num_x), 바이어스는 1로 설정하고 은닉층의 초깃값(num_yh), 출력층의 초깃값(num_y0)도 지정<br>
self.activation~을 통해 활성화 함수 초깃값을 지정했고 self.weight_in,out을 통해 가중치 입력 초깃값, 출력 초깃값 지정!<br>
random.random()은 0~1사이의 실수(float)을 리턴해준다.<br>
self.gradient_in,out은 모멘텀 SGD를 위한 이전 가중치 초깃값이다.

<h2> 역전파 실행 함수 </h2><br>
델타 출력 계산, 은닉 노드의 오차 함수, 출력 가중치 업데이트, 입력 가중치 업데이트 , 최소 제곱법을 이용한 오차 계산


In [68]:
if __name__ == '__main__':
    n = NeuralNetwork(2,2,1)
    n.train(data)
    n.result(data)

Error : 0.99064
Error : 0.00398
Error : 0.00327
Error : 0.00302
Error : 0.00290
Error : 0.00282
Error : 0.00277
Error : 0.00273
Error : 0.00270
Error : 0.00268
Input: [0, 0], Predict : [0.0008096063430629609]
Input: [0, 1], Predict : [0.9489496010872271]
Input: [1, 0], Predict : [0.9479865083465]
Input: [1, 1], Predict : [0.0005448532667056171]


오차 역전파를 함수로 설명했으므로 입력값은 그대로 하고 타깃 값을 OR,AND,NAND에 맞게 설정해주면 Predict도 정해진 타깃 값에 따라 정해진다!